In [ ]:
import tensorflow as tf
import keras

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

try:
    for i in gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

except Exception as e:
    print(f"{e}")

In [ ]:
BATCH_SIZE: int = 8
SEED: int = 123
LABEL_MODE: str = 'int'
IMAGE_SIZE: tuple = (224, 224)
INTERPOLATION: str = "bilinear"
COLOR: str = 'rgb'
LABELS: str = "inferred"

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
path = "/mnt/d/Tojo Sir - Project/"
train_path = path + "isbi2025-ps3c-train-dataset/"
test_path = path + "isbi2025-ps3c-test-dataset/"

In [ ]:
try:
    train_set = keras.utils.image_dataset_from_directory(
        train_path,
        batch_size = BATCH_SIZE,
        seed = SEED,
        subset = "training",
        validation_split = 0.3,
        shuffle = True,
        interpolation = INTERPOLATION,
        label_mode = LABEL_MODE,
        color_mode = COLOR,
        labels = LABELS,
        image_size = IMAGE_SIZE 
    )

except:
    print("Error loading training set\n")

In [ ]:
try:
    valid_set = keras.utils.image_dataset_from_directory(
        train_path,
        batch_size = BATCH_SIZE,
        seed = SEED,
        subset = "validation",
        shuffle = False,
        validation_split = 0.3,
        label_mode = LABEL_MODE,
        color_mode = COLOR,
        labels = LABELS,
        image_size = IMAGE_SIZE 
    )

except:
    print("Error loading Validation set")

In [ ]:
def normalizeImages(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
train = train_set.map(normalizeImages, num_parallel_calls = AUTOTUNE).cache('train_cache').prefetch(buffer_size = AUTOTUNE)
valid = valid_set.map(normalizeImages, num_parallel_calls = AUTOTUNE).cache('valid_cache').prefetch(buffer_size = AUTOTUNE) 

In [ ]:
data_augmentation = keras.models.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomZoom(0.1, 0.1),
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomRotation(0.2)
], name = "data_augmentation")

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input((IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),

    #data_augmentation,

    keras.layers.Conv2D(16, (2,2), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(32, (2,2), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(64, (2,2), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Flatten(),

    keras.layers.Dense(1024, activation = keras.activations.leaky_relu),
    keras.layers.Dense(256, activation = keras.activations.leaky_relu),
    keras.layers.Dense(64, activation = keras.activations.leaky_relu),
    keras.layers.Dense(4, activation = keras.activations.softmax)
])

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'],
              jit_compile=True)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [ ]:
model.fit(train,  validation_data = valid, epochs = 256, callbacks=[early_stopping])